# Imports

In [20]:
#!pip install torchtext
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
import torchtext
import torch
from torch import nn,optim
from time import time

In [39]:
data = pd.read_csv("dataframe.csv")
print(data)



      Unnamed: 0                                               text  encoded
0              0  The soil I’m walking over comes    from deeper...    False
1              1  the close air of the earth whence she derived ...     True
2              2  Lyric night of the lingering Indian Summer,\nS...    False
3              3  “Percussus sum sicut foenum, et aruit cor meum...    False
4              4  I should be happy with my lot: A wife and moth...    False
...          ...                                                ...      ...
4789        4789  office has never really is its oldest daughter...     True
4790        4790  soot from her car with suitcases and hugged he...     True
4791        4791  Seeing in crowded restaurants the one you love...    False
4792        4792  flower sweet as I can never forsake And Fortun...     True
4793        4793  Echo that loved hid within a wood Would to her...    False

[4794 rows x 3 columns]


Creating tokenizer and vocabulary

In [40]:
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.nn.utils import rnn
from sklearn.model_selection import train_test_split

data, test = train_test_split(data, test_size=0.9)
train, test = train_test_split(data, test_size=0.2)


tokenizer = get_tokenizer("basic_english")

def build_vocabulary(datasets):
    for dataset in datasets:
        #print(dataset)
        for text in dataset['text']:
            yield tokenizer(text)

vocab = build_vocab_from_iterator(build_vocabulary([data,data]), min_freq=3, specials=["<UNK>"])

vocab.set_default_index(vocab["<UNK>"])

for l in data['text']:
  tokens = tokenizer(l)
  indexes = vocab(tokens)


Creating Dataloader

In [41]:

class Text(Dataset):
  def __init__(self,df):
    self.df = df
    #print(df)

  def pad(self,seq):
    if len(seq)>140:
      return seq[:140]
    else:
      for i in range(140-len(seq)):
        seq.insert(0,0)
      return seq

  def __len__(self):
    return len(self.df['encoded'])
  def __getitem__(self,idx):

    return torch.tensor(self.pad(vocab(tokenizer(self.df['text'][idx])))) , int(self.df['encoded'][idx])

test_loader  = DataLoader(Text(test.reset_index()), batch_size=4)
train_loader = DataLoader(Text(train.reset_index()),batch_size=4)

for txt , tgt in test_loader:
  _
  #print(txt)
  #print(txt)
voc = len(vocab)
#print(tokens)

print(vocab(["<UNK>"]))

[0]


In [42]:
print(test)
print(train)

      Unnamed: 0                                               text  encoded
4592        4592  weals down. Autumn at dusk and file I come nea...     True
4487        4487  s tanning lotion. Dearer far Even than all lov...     True
2242        2242  from An Evening's Love    You charm'd me not w...    False
2632        2632  Titled after Satie    I. Three pears ripen On ...    False
1745        1745  I will arise and go now, and go to Innisfree, ...    False
...          ...                                                ...      ...
376          376  his darts this soup. Beholding choice molecule...     True
3903        3903  One cuts blocks  From the abundant river,  Hau...    False
3014        3014  ; ; ; ; ; ; ; ; ; ; ; ; ;~ [our] nightmare : n...    False
3898        3898  Gaily bedight,    A gallant knight, In sunshin...    False
4571        4571  He never saw a violin. But he saw a lifetime o...    False

[96 rows x 3 columns]
      Unnamed: 0                                     

## Utils


In [43]:
def bitwise_input(input,n_max = 16):
    bits = []
    for i in bin(input)[2:]:
        bits.append(int(i))
    # filling
    for _ in range(n_max - len(bits)):
        bits.insert(0,0)
    return bits[:n_max]

print(bitwise_input(1,2))

[0, 1]


In [44]:
def getProb(prob,nmax = 16):
    out = []
    for i in range(nmax):
        out.append(prob[2**nmax-1])
    print(out)
    return out




# Quantum Layer

In [45]:
import pennylane as qml
n_qubits = 4
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def qnode(inputs, weights):
    #inp = bitwise_input(inputs,n_qubits)
    #print(inputs)
    qml.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.BasicEntanglerLayers(weights, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(wires = i)) for i in range(n_qubits)]


In [46]:
n_layers = 1
weight_shapes = {"weights": (n_layers, n_qubits)}

In [47]:
class HybridModel(torch.nn.Module):


    def __init__(self , embeddingSize):
        super().__init__()
        self.memory = {}
        self.inpFormat = torch.nn.Linear(16, n_qubits*2)
        self.qlayer_1 = qml.qnn.TorchLayer(qnode, weight_shapes)
        self.qlayer_2 = qml.qnn.TorchLayer(qnode, weight_shapes)
        self.clayer = torch.nn.Linear(2*n_qubits, embeddingSize)



    def __EmbeddingBag__(self,X):

        if str(X) in self.memory:
            # Check if index is Cached
            return self.memory[str(X)].clone()
        #Formatting Index into Binary
        x = torch.tensor(bitwise_input(int(X) , 16))
        #Convert Binary output into float
        x = self.inpFormat(x.float())
        #Split Layer into 2 seperate Quantum Circuits
        x_1 = self.qlayer_1(x[:n_qubits-1])
        x_2 = self.qlayer_2(x[n_qubits :])


        #Concatnate and Resize output
        xc = torch.cat([x_1 , x_2], axis=-1)
        xc = self.clayer(xc.float())

        #Cache Value
        self.memory[str(X)] = xc.clone()
        return xc

    def __RecursiveBag__(self,x):
        #print(x.dim())
        if x.dim()==0: return self.__EmbeddingBag__(x)

        Bag = []
        for subtensor in x:
            Bag.append(self.__RecursiveBag__(subtensor))
        #print(Bag)
        return torch.stack(Bag)

    def forward(self, x):
        self.memory = {}
        if  x.dim()==0: return self.__EmbeddingBag__(x)
        return self.__RecursiveBag__(x)

qmodel = HybridModel(64)


In [48]:
print(qmodel)
print(qmodel(torch.tensor([13])))

# Testing Hybrid Model for Consistency

In [49]:
from random import randint
import seaborn as sb

In [50]:
testrange = 32
out = []
inp = -1
#for i in range(testrange):
    #inp +=1
    #print(inp)
    #X = torch.tensor([inp])
    #out.append(qmodel(X).detach().numpy().reshape([64]).tolist())

#sb.heatmap(out,cmap= sb.color_palette("coolwarm"))


In [51]:
#for i in qmodel.named_parameters():
    #print(i)

# Declaring Hybrid Model

In [52]:

class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()
        self.embedding_layer = HybridModel(64)
        self.rnn = nn.RNN(64,32, 3, batch_first=True)
        self.linear1 = nn.Linear(32,32)
        self.linear2 = nn.Linear(32,2)
        self.act = nn.ReLU()
        self.out = nn.LogSoftmax(dim = 1)

    def forward(self, X_batch):
        embeddings = self.embedding_layer(X_batch)
        output, hidden = self.rnn(embeddings)
        return self.out(self.linear2(self.act(self.linear1(self.act(output[:,-1])))))

#model = RNN()

print(model)


RNN(
  (embedding_layer): HybridModel(
    (inpFormat): Linear(in_features=16, out_features=8, bias=True)
    (qlayer_1): <Quantum Torch Layer: func=qnode>
    (qlayer_2): <Quantum Torch Layer: func=qnode>
    (clayer): Linear(in_features=8, out_features=64, bias=True)
  )
  (rnn): RNN(64, 32, num_layers=3, batch_first=True)
  (linear1): Linear(in_features=32, out_features=32, bias=True)
  (linear2): Linear(in_features=32, out_features=2, bias=True)
  (act): ReLU()
  (out): LogSoftmax(dim=1)
)


In [53]:
from word2ket import EmbeddingKet, EmbeddingKetXS , ketify,summary
summary(model)

Module Name                                                                           Total Parameters  Trainable Parameters # Elements in Trainable Parametrs       
Linear(in_features=16, out_features=8, bias=True)                                     2                 2                    136                                     
<Quantum Torch Layer: func=qnode>                                                     1                 1                    4                                       
<Quantum Torch Layer: func=qnode>                                                     1                 1                    4                                       
Linear(in_features=8, out_features=64, bias=True)                                     2                 2                    576                                     
RNN(64, 32, num_layers=3, batch_first=True)                                           12                12                   7,360                                   
Line

9202

In [54]:
accuracy = []
optimizer = optim.SGD(model.parameters(), lr=0.017, momentum=0.9)
criterion = nn.NLLLoss()
time0 = time()

Training

In [55]:
import tqdm

In [56]:
accuracy = []
epochs = 1
for e in range(epochs):
    running_loss = 0
    with tqdm.tqdm(train_loader , unit = "batch") as tepoch:
      for text, tgt in tepoch:
        tepoch.set_description(f"Epoch {e}")

        with torch.autograd.set_detect_anomaly(True):
          optimizer.zero_grad()
          output = model.embedding_layer(text)
          loss = criterion(output, tgt)
          loss.backward(retain_graph=True)

        optimizer.step()
        tepoch.set_postfix(loss=loss.item())
        #print(loss.item())
        running_loss += loss.item()

      else:
        #print("\nTraining Time (in minutes) =",(time()-time0)/60)
        #print(output)

        print("\nEpoch {} - Training loss: {}".format(e+1, running_loss/len(test_loader)))


    correct_count, all_count = 0, 0
    for images,labels in test_loader:
      for i in range(len(labels)):
        img = images[i].view(1,-1)
        with torch.no_grad():
          logps = model(img)


        ps = torch.exp(logps)
        probab = list(ps.numpy()[0])
        pred_label = probab.index(max(probab))
        true_label = labels.numpy()[i]
        #print(pred_label)
        if(true_label == pred_label):
          correct_count += 1
        all_count += 1

    #print("Number Of Images Tested =", all_count)
    print("Model Accuracy =", (correct_count/all_count))
    accuracy.append((correct_count/all_count))


Epoch 0:   0%|          | 0/96 [00:00<?, ?batch/s]

Epoch 0:  15%|█▍        | 14/96 [3:51:05<22:33:29, 990.36s/batch, loss=0.733]


KeyboardInterrupt: 

#Texts for testing:

Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's

Even my brother is not like to speak with me. They treat me like aids patent.

